# Vehicle Detection & Tracking
Udacity Project 5
Author: Kelly Smith

This Jupyter Notebook will train a classifier to identify vehicles using classic computer vision techniques (HOG, color histograms, sliding windows). 

In [47]:
# Import statements to get all requisite packages
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import pickle
import os.path
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog

In [48]:
# Define constants for the pipeline
trainingDataFilename = "trainingData.p"

In [2]:
"""
This will be the main function for processing an image, detecting and localizing the vehicles.
"""
def processImg(img):
    pass

In [59]:
"""
This is the test function for executing the overall pipeline on a single image.
"""
def test():
    
    trainClassifier(Debug=True)

test()

Found existing training data.  Loading that to save time...Complete!
Training the classifier...Complete!
Test Accuracy of SVC =  0.9285


In [58]:
"""
This function will train a linear SVM from the sklearn library on the car and non-car images
"""
def trainClassifier(Debug=False):
    # Instantiate the linear support vector classifier
    svc = LinearSVC()
    
    # If the training data has not been generated yet,
    if not os.path.isfile(trainingDataFilename):
        # Build the training data.
        if Debug:
            print("No training data detected.  Building training data from scratch (hang tight for a minute)...", end='')
        X_train, y_train, X_test, y_test = buildTrainingData()
        if Debug:
            print("Finished building training data!")
    else:
        # Load the existing data.
        if Debug:
            print("Found existing training data.  Loading that to save time...",end='')
        X_train, y_train, X_test, y_test = loadTrainingData()
        if Debug:
            print("Complete!")
    
    if Debug:
        print("Training the classifier...",end='')
    # Train the classifier on the training set.
    svc.fit(X_train, y_train)
    if Debug:
        print("Complete!")
    
    # Evaluate fit on test set
    if Debug:
        print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
    
    return svc

In [20]:
"""
This function will perform a sliding window search over the image to identify vehicles.
"""
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    if x_start_stop == [None, None]:
        x_start_stop = [0, img.shape[1]]
    if y_start_stop == [None, None]:
        y_start_stop = [0, img.shape[0]]
    
    spanx = x_start_stop[1] - x_start_stop[0]
    spany = y_start_stop[1] - y_start_stop[0]
    
    dx = np.int((1-xy_overlap[0]) * xy_window[0])
    dy = np.int((1-xy_overlap[1]) * xy_window[1])
    
    nx = np.int(spanx / dx) - 1
    ny = np.int(spany / dy) - 1

    # Initialize a list to append window positions to
    window_list = []
    for i in range(nx):
        for j in range(ny):
            startx = x_start_stop[0] + i*dx
            starty = y_start_stop[0] + j*dy
            windowPos = ((startx, starty),(startx+xy_window[0], starty+xy_window[1]))
            window_list.append(windowPos)
            # window_list.append( ((startx, starty), (endx, endy)))
    # Loop through finding x and y window positions
    #     Note: you could vectorize this step, but in practice
    #     you'll be considering windows one by one with your
    #     classifier, so looping makes sense
        # Calculate each window position
        # Append window position to list
    # Return the list of windows
    return window_list

In [29]:
"""Test routine for sliding window algorithm."""
def test_sliding_window():
    img = mpimg.imread("test_images/test1.jpg")
    windows = slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(128, 128), xy_overlap=(0.75, 0.75))
                       
    window_img = draw_boxes(img, windows, color=(0, 0, 255), thick=6)                    
    plt.imshow(window_img)
    plt.show()
    
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy
    
# test_sliding_window()

In [60]:
"""This function will prepare features and labels for training the classifier."""
def buildTrainingData(spatial=16, histbin=32, SaveData=True, Debug=False):
    # Get car features
    cars = glob.glob("vehicles/*/*.png")
    notcars = glob.glob("non-vehicles/*/*.png") 
    
    if Debug:
        print("There are %d cars examples." % (len(cars)))
        print("There are %d not-car examples." % (len(notcars)))
        print("Extracting color features for cars...",end="")
    car_features = extract_color_features(cars, cspace='RGB', spatial_size=(spatial, spatial),
                            hist_bins=histbin, hist_range=(0, 256))
    if Debug:
        print("Complete!")
        print("Extracting color features for non-cars",end="")
    # Get not-car features.
    notcar_features = extract_color_features(notcars, cspace='RGB', spatial_size=(spatial, spatial),
                            hist_bins=histbin, hist_range=(0, 256))
    if Debug:
        print("Complete!")
    
    
    # Create an array stack of feature vectors
    X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)

    # Define the labels vector
    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(
        scaled_X, y, test_size=0.2, random_state=rand_state)

    # If the SaveData flag is True, then save the data via pickle 
    # to avoid having to generate training data every time.
    if SaveData:
        saveTrainingData(X_train, y_train, X_test, y_test)
        
    return (X_train, y_train, X_test, y_test)

"""This function will save the training data via pickle"""
def saveTrainingData(X_train, y_train, X_test, y_test):
    data = {"X_train":X_train, "y_train":y_train, "X_test":X_test, "y_test":y_test}
    with open (trainingDataFilename,"wb") as output_file:
        pickle.dump(data, output_file)
        print("Saved the training data to %s" % trainingDataFilename)
    
"""This function loads existing training data."""
def loadTrainingData():
    with open (trainingDataFilename,"rb") as input_file:
        data = pickle.load(input_file)
    return (data['X_train'], data['y_train'], data['X_test'], data['y_test'])

# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_color_features(imgs, cspace='RGB', spatial_size=(32, 32),
                        hist_bins=32, hist_range=(0, 256)):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        else: feature_image = np.copy(image)      
        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        # Apply color_hist() also with a color space option now
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
        # Append the new feature vector to the features list
        features.append(np.concatenate((spatial_features, hist_features)))
    # Return list of feature vectors
    return features

In [15]:
"""
This function returns a histogram of oriented gradients
"""
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

In [17]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_hog_features(imgs, cspace='RGB', orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        # Append the new feature vector to the features list
        features.append(hog_features)
    # Return list of feature vectors
    return features

In [35]:
def getCarData():
    # Read in car and non-car images
    car_images = glob.glob('vehicles/*/*.png')
    not_car_images = glob.glob('non-vehicles/*/*.png')
    cars = []
    notcars = []
    
    print("There are %d vehicle images." % (len(car_images)))
    print("There are %d non-vehicle images." % (len(not_car_images)))
getCarData()

There are 8792 vehicle images.
There are 8968 non-vehicle images.
